In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import seaborn as sns
import scienceplots

plt.style.use(['science', 'ieee'])
plt.rcParams.update({"font.family": "sans", "font.serif": [
                    "cm"], "mathtext.fontset": "cm", "font.size": 16})

def generate_calibration_diagrams_sns(screen_width=1920, screen_height=1080, num_points=50, error_margin=150):
    sns.set(style="whitegrid")

    # Generate random gaze points
    x = np.random.randint(0, screen_width, num_points)
    y = np.random.randint(0, screen_height, num_points)
    points = np.column_stack((x, y))

    # Calculate convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Function to plot screen boundaries
    def plot_screen_boundaries():
        plt.plot([0, screen_width, screen_width, 0, 0], 
                 [0, 0, screen_height, screen_height, 0], 'k--', linewidth=1.5)

    # Function to remove axis numbers
    def remove_axis_numbers():
        plt.gca().set_xticklabels([])
        plt.gca().set_yticklabels([])

    # 1. Convex Hull of Captured Gaze Points
    plt.figure(figsize=(10, 6), dpi=300)
    sns.scatterplot(x=x, y=y, color='blue', s=40, label='Gaze Points')
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'r-', linewidth=2)
    plot_screen_boundaries()
    remove_axis_numbers()
    plt.title('Convex Hull of Captured Gaze Points', fontsize=16)
    plt.xlabel('Screen Width (pixels)', fontsize=14)
    plt.ylabel('Screen Height (pixels)', fontsize=14)
    plt.legend(fontsize=12)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.tight_layout(pad=1.0)
    plt.savefig('convex_hull_high_quality_sns.png', bbox_inches='tight')
    plt.close()

    # 2. Error Margin Extension
    plt.figure(figsize=(10, 6), dpi=300)
    sns.scatterplot(x=x, y=y, color='blue', s=40, label='Gaze Points')
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'r-', linewidth=2)
    
    # Calculate and plot extended hull
    center = np.mean(hull_points, axis=0)
    extended_hull_points = hull_points + error_margin * (hull_points - center) / np.linalg.norm(hull_points - center, axis=1)[:, np.newaxis]
    extended_hull_points = np.vstack((extended_hull_points, extended_hull_points[0]))  # Close the polygon
    plt.plot(extended_hull_points[:, 0], extended_hull_points[:, 1], 'g--', linewidth=2, label='Error Margin')
    
    plot_screen_boundaries()
    remove_axis_numbers()
    plt.title('Error Margin Extension', fontsize=16)
    plt.xlabel('Screen Width (pixels)', fontsize=14)
    plt.ylabel('Screen Height (pixels)', fontsize=14)
    plt.legend(fontsize=12)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.tight_layout(pad=1.0)
    plt.savefig('error_margin_extension_high_quality_sns.png', bbox_inches='tight')
    plt.close()

    # 3. Final Calibration Points
    plt.figure(figsize=(10, 6), dpi=300)
    plot_screen_boundaries()
    
    # Clip extended hull to screen boundaries
    clipped_hull = np.clip(extended_hull_points, [0, 0], [screen_width, screen_height])
    plt.plot(clipped_hull[:, 0], clipped_hull[:, 1], 'g-', linewidth=2, label='Clipped Extended Hull')
    
    # Find corners of clipped hull
    min_x, min_y = np.min(clipped_hull, axis=0)
    max_x, max_y = np.max(clipped_hull, axis=0)
    corners = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]])
    
    # Plot corners
    sns.scatterplot(x=corners[:, 0], y=corners[:, 1], color='red', s=100, marker='x', label='Calibration Points')
    
    for i, (x, y) in enumerate(corners):
        plt.annotate(f'Point {i+1}', (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10)
    
    remove_axis_numbers()
    plt.title('Final Calibration Points', fontsize=16)
    plt.xlabel('Screen Width (pixels)', fontsize=14)
    plt.ylabel('Screen Height (pixels)', fontsize=14)
    plt.legend(fontsize=12)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.tight_layout(pad=1.0)
    plt.grid(True)
    plt.savefig('final_calibration_points_high_quality_sns.png', bbox_inches='tight')
    plt.close()

# Generate the diagrams using Seaborn
generate_calibration_diagrams_sns()